In [ ]:
# Imports
import sys
MIN_REQ_PYTHON = (3,9)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

import requests
import datetime

from requests.auth import HTTPBasicAuth

import json
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
# OPTION 1 : import the key api named 'bitsight' from the keyring manager
# below the python cli (to execute once) create the api key : 
# >>> import keyring
# >>> import readline
# >>> keyring.set_password(service_name="bitsight",username="None",password="xxx")
# >>> readline.clear_history()
import keyring
keyring.get_keyring()
bitsight_apikey = keyring.get_credential("bitsight", "None")

In [ ]:
# OPTION 2 : import the key api named from the environment variable 
import os
bitsight_apikey = os.environ['BITSIGHT_API_KEY']

In [ ]:
### PORTFOLIO: REQUEST A JSON WITH THE SUBSIDIARIES AND THE RELATIVE GUID ###
bitsight_portfolio_url = "https://api.bitsighttech.com/ratings/v2/portfolio"
jsonResponse = []

try:
    res = requests.post(url=bitsight_portfolio_url, verify=True, auth = HTTPBasicAuth(bitsight_apikey, ''))
    jsonResponse = res.json()

except requests.exceptions.SSLError as ssl_error:
    inner_exception = ssl_error.args[0]
    inner_ssl_error = inner_exception.reason.args[0]
    if str(inner_ssl_error).find('SSL: CERTIFICATE_VERIFY_FAILED') == -1:
        for key, value in vars(inner_ssl_error).items():
            dnshost = value['subject'][0][0][1]
            is_ssl_error = True
            for alt in value['subjectAltName']:
                if alt[0]=="DNS":
                    san.append(alt[1])
                pass
            
except Exception as e:
    print(type(e))
    print(e)
                
except requests.exceptions.RequestException as e:
    print("Error: ", e)
    pass
            
except urllib3.exceptions as e:
    print("Error: ", e)
    pass
            
except socket.gaierror as e:
    print("Error: ", e)
    pass

In [ ]:
### PORTFOLIO: DISPLAY THE JSON RESPONSE
jsonResponse

In [ ]:
### PORTFOLIO: CREATE A DATAFRAME TO MAKE FURTHER OPERATIONS / REQUESTS ###
jsonResponse = res.json()
df1 = pd.json_normalize(
    jsonResponse,
    record_path=[['results'][0]])

subsidiaries = pd.DataFrame(df1, columns=['name', 'guid', 'rating', 'rating_date'])
subsidiaries.set_index('name', inplace=True)
subsidiaries

In [ ]:
### PORTFOLIO : EXAMPLE OF OPERATION : removing companies that are not subsidiaries 
subsidiaries.drop('Kering, Inc - Luxury Brands', inplace=True)
subsidiaries.drop('Bank of America Corporation', inplace=True)
subsidiaries.drop('SurveyMonkey Corporation', inplace=True)
subsidiaries

In [ ]:
### CSV VULNERABILITIES REPORT : ONE SUBSIDIARY 1/2
company_guid = input("Enter the company GUID: ")
bitsight_findings = "https://api.bitsighttech.com/v1/companies/" + company_guid + "/findings"
jsonResponse = []

try:
    res = requests.post(url=bitsight_findings, verify=True, auth = HTTPBasicAuth(bitsight_apikey, '')) 
    jsonResponse = res.json()

except requests.exceptions.SSLError as ssl_error:
    inner_exception = ssl_error.args[0]
    inner_ssl_error = inner_exception.reason.args[0]
    if str(inner_ssl_error).find('SSL: CERTIFICATE_VERIFY_FAILED') == -1:
        for key, value in vars(inner_ssl_error).items():
            dnshost = value['subject'][0][0][1]
            is_ssl_error = True
            for alt in value['subjectAltName']:
                if alt[0]=="DNS":
                    san.append(alt[1])
                pass
            
except Exception as e:
    print(type(e))
    print(e)
                
except requests.exceptions.RequestException as e:
    print("Error: ", e)
    pass
            
except urllib3.exceptions as e:
    print("Error: ", e)
    pass
            
except socket.gaierror as e:
    print("Error: ", e)
    pass

In [ ]:
### CSV VULNERABILITIES REPORT : ONE SUBSIDIARY 2/2
vuln_report = pd.DataFrame(columns=['IP', 'port', 'CVE', 'CVSS','confidence'])
iloc = 0

for finding in jsonResponse['results']:   
    for asset in finding['assets']:
        if "vulnerabilities" in finding['details']:
            for vuln in finding['details']['vulnerabilities']:
                row = [asset['asset'],finding['details']['dest_port'],vuln['name'],vuln['cvss']['base'],vuln['confidence']]
                vuln_report.loc[iloc] = row
                iloc = iloc +1

vuln_report

In [ ]:
from pandasgui import show
gui = show(vuln_report)

In [ ]:
### CSV VULNERABILITIES REPORT : ONE SUBSIDIARY DEBUG
jsonResponse

In [ ]:
### CSV VULNERABILITIES REPORTS : FULL EXTRACT
# DF that will be saved as CSV at the end of the iteration
df_vulns = pd.DataFrame(columns=['asset', 'port', 'CVE', 'CVSS','confidence'])
    
# get_findings(): Get the bitsight findings in a json format, using requests lib
# parameter 'sub': GUID of the subsidiary (got from the portfolio)
def get_findings(sub, df_vulns):

    bitsight_findings_url = "https://api.bitsighttech.com/v1/companies/" + sub + "/findings"

    try:
        res = requests.post(url=bitsight_findings_url, verify=True, auth = HTTPBasicAuth(bitsight_apikey, ''))
        jsonResponse = res.json()
        print("Count: ",jsonResponse['count'])
        print(bitsight_findings_url)
        df_vulns = get_vulns(jsonResponse, df_vulns)
        bitsight_findings_url_next = jsonResponse['links']['next']

        # while loop to iterate over the findings (if more than 100)
        # get the vulnerabilities and append into the DF get_vulns
        while bitsight_findings_url_next is not None:    

            print(bitsight_findings_url_next)
            try:
                res = requests.post(url=bitsight_findings_url_next, verify=True, auth = HTTPBasicAuth(bitsight_apikey, ''))
                jsonResponse = res.json()
                df_vulns = get_vulns(jsonResponse, df_vulns)
                bitsight_findings_url_next = jsonResponse['links']['next']
            
            except Exception as e:
                print(type(e))
                print(e)

    except Exception as e:
        print(type(e))
        print(e)
            
# get_vulns(): Get the vulnerabilities from a findings page, append it to the DF vuln_report
# parameter 'jsonResponse': json page of findings (default limit/offset is 100)
# parameter 'vuln_report': DF used to store all vulnerabilities related to a subsidiary 
def get_vulns(jsonResponse,df_vulns):
    iloc = len(df_vulns)
    x = 0
        
    while x < len(jsonResponse['results']):  
        for asset in jsonResponse['results'][x]['assets']:
            if "vulnerabilities" in jsonResponse['results'][x]['details']:
                for vuln in jsonResponse['results'][x]['details']['vulnerabilities']:
                    if "dest_port" in jsonResponse['results'][x]['details']:
                        row = [asset['asset'],jsonResponse['results'][x]['details']['dest_port'],vuln['name'],vuln['cvss']['base'],vuln['confidence']]
                    else:
                        row = [asset['asset'],'',vuln['name'],vuln['cvss'],vuln['confidence']]
                    df_vulns.loc[iloc] = row
                    iloc = iloc +1
        x = x +1
    
    return df_vulns

# foreach subsidiary in the portfolio: 
# 1/ get the vulns from the finding (while loop to pagination)
# 2/ save it in a dedicated CSV
for index,sub in subsidiaries['guid'].iteritems():

    df_vulns_report = pd.DataFrame(columns=['asset', 'port', 'CVE', 'CVSS','confidence'])
    get_findings(sub, df_vulns_report)
    
    # write the DF get_vulns in a CSV 
    pd.set_option('display.max_colwidth', None)            
    current_date = datetime.datetime.now()
    cdate = str(current_date.year) + str(current_date.month) + str(current_date.day)
    input_file = "bitsight_" +  cdate +"_"+ sub  
    df_vulns_report.to_csv(str(input_file + '.csv'))